–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

<img style="float: right; margin: 35px 35px 1px 1px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" width="160px" height="135px"/>

# Proyecto Final: Ecuación de Black-Scholes 

## Ingeniería Financiera - Decisiones y Teoría de Juegos 

### Maestro: Juan Martín Casillas González

### 8 de mayo de 2023.

> **Por:** 
> - Gustavo Adrián Herrera Barragán
> - José Alfonso Martínez Ramirez
> - Erick Uribe Rodriguez

–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

## 0. Introducción Librerías:


In [12]:
import numpy as np
import scipy as scp
import scipy.stats as ss
import scipy.stats as st
import scipy.linalg as linalg
from scipy import sparse
from scipy.integrate import quad
from scipy.sparse.linalg import spsolve
from functools import partial
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline


–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

## 1. Valor de una opcion con el Método Implícito de Diferencias Finitas

In [13]:
S = 60
Smax = 100
K = 50
r = 0.01
T = 12/12
sigma = 0.4
is_call = True
M = 100
N = 100

In [14]:
def FD_Vanilla_Option(S, K, r, T, sigma, Smax, M, N, is_call=True):
    '''Calcula el valor de la opcion Vanilla usando el
    metodo de diferencias finitas
    
    Parametros
    ----------
    S: precio spot
    K: precio strike
    T: tiempo de vencimiento
    r: tasa de interes libre de riesgo
    sigma: volatilidad del activo
    Smax: maximo valor del activo
    M: numero de particiones del activo
    N: numero de particiones del tiempo
    is_call: define si es opcion call o put
    
    Resultado
    -------
    V : Valor de la opcion
    '''
    M, N = int(M), int(N)  

    dS = Smax / float(M)
    dt = T / float(N)

    grid = np.zeros((M+1, N+1))
    iValues = np.arange(M)
    jValues = np.arange(N)
    SValues = np.linspace(0, Smax, M+1)

    if is_call == True:
        '''
        Condiciones iniciales opcion Call:
        C(0,t) = 0
        C(Smax,t) = Smax - K e^(-r(T-t))
        C(S,T) = Smax - K
        '''
        grid[:,-1] = np.maximum(SValues - K, 0)
        grid[-1,:-1] = Smax - K * np.exp(-r * dt * (N - jValues))
    else:
        '''
        Condiciones iniciales:
        P(0,t) = K e^(-r(T-t))
        P(Smax,t) =  0
        P(S,T) = K - Smax
        '''
        grid[:, -1] = np.maximum(K - SValues, 0)
        grid[0, :-1] = K * np.exp(-r * dt * (N - jValues))

    alpha =  0.5 * dt * iValues * (r  - sigma**2 * iValues)
    beta  =  1 + dt * (r + sigma**2 * iValues**2)
    gamma = -0.5 * dt * iValues * (r  + sigma**2 * iValues)

    coeffs = np.diag(alpha[2:M], -1) + np.diag(beta[1:M]) + np.diag(gamma[1:M-1], 1)

    P, L, U = linalg.lu(coeffs)
    aux = np.zeros(M-1)

    for j in reversed(range(N)):
        aux[0] = np.dot(alpha[1], grid[0, j])
        aux[-1] = np.dot(-gamma[M-1], grid[M, j])
        x1 = linalg.solve(L, grid[1:M, j+1]+aux)
        x2 = linalg.solve(U, x1)
        grid[1:M, j] = x2
    
    V = np.interp(S, SValues, grid[:, 0])
    return V

call_1 = FD_Vanilla_Option(S, K, r, T, sigma, Smax, M, N, is_call=True)
put_1 = FD_Vanilla_Option(S, K, r, T, sigma, Smax, M, N, is_call=False)
print('El valor de la opcion call es :', call_1)
print('El valor de la opcion put es :', put_1)

El valor de la opcion call es : 14.876691720457153
El valor de la opcion put es : 4.379206573391193


## 1. Valor de una opcion con el Método Analítico

In [15]:
def BS_Vanilla_Option(S, K, r, T, sigma, is_call = True):
    '''Calcula el valor de la opcion Vanilla usando el
    metodo de diferencias finitas
    
    Parametros
    ----------
    S: precio spot
    K: precio strike
    T: tiempo de vencimiento
    r: tasa de interes libre de riesgo
    sigma: volatilidad del activo
    
    Resultado
    -------
    V : Valor de la opcion
    '''
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))
    if is_call == True:
        V = (S * st.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * st.norm.cdf(d2, 0.0, 1.0))
    else:
        V = K * np.exp(-r * T) * st.norm.cdf(-d2, 0.0, 1.0) - S * st.norm.cdf(-d1, 0.0, 1.0) 
    return V

call_2 = BS_Vanilla_Option(S, K, r, T, sigma, is_call=True)
put_2 = BS_Vanilla_Option(S, K, r, T, sigma, is_call=False)

print('El valor de la opcion call es :', call_2)
print('El valor de la opcion put es :', put_2)

El valor de la opcion call es : 14.895416292915225
El valor de la opcion put es : 4.3979079803736205


<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Creado con Jupyter por: Gustavo Adrián Herrera Barragán, José Alfonso Martínez Ramirez & Erick Uribe Rodriguez.
</footer>